# EEG - Flow

## 10. ICA apply
This is a copy paste of step 7 with the difference we now interpolate bad channels for the further frequency analyses

Last edit: 30.09.2024 15:13

In [8]:
import multiprocessing as mp
import time
from itertools import product

from mne import set_log_level as set_log_level_mne

from eeg_flow import set_log_level
from eeg_flow.utils import parallel
from eeg_flow.tasks import apply_ica_interpolate

In [9]:
set_log_level_mne("WARNING")
set_log_level("INFO")

The parameters of the file to process are defined below. Locks are created to prevent someone else from running the same task and from writing the same derivatives.

In [14]:
# PARTICIPANTS_WITH_GROUPS: list[str] = ["P02-G2","P04-G3", "P05-G2", "P07-G5", "P08-G6", 
#     "P10-G8", "P11-G1", "P12-G2", "P13-G3", "P14-G3", "P15-G4", "P16-G4", "P18-G6", "P19-G6", 
#     "P21-G7", "P22-G7", "P23-G7", "P25-G1", "P26-G2", "P27-G8", "P30-G4", "P31-G5", "P32-G6",
#     "P33-G7", "P53-G8", "P55-G1", "P58-G5", "P59-G8", "P60-G6", "P61-G4", "P62-G3", "P63-G5", 
#     "P65-G1", "P66-G5", "P67-G3", "P69-G8", "P72-G1", "P73-G3", "P74-G1", "P75-G1"]
# len(PARTICIPANTS_WITH_GROUPS)

40

In [16]:
# PARTICIPANTS_WITH_GROUPS: list[str] = ["P34-G8","P35-G1","P36-G3","P37-G2","P38-G8","P40-G8","P42-G5","P43-G4","P44-G6","P45-G2","P46-G3","P47-G7","P48-G8","P49-G1","P50-G2","P51-G3"]  # list of str "Pxx-Gy", e.g. ["P02-G2"]
PARTICIPANTS_WITH_GROUPS: list[str] = [ "P05-G2", "P07-G5", "P08-G6", 
     "P10-G8", "P11-G1", "P12-G2", "P13-G3", "P14-G3", "P15-G4", "P16-G4", "P18-G6", "P19-G6", 
     "P21-G7", "P22-G7", "P23-G7", "P25-G1", "P26-G2", "P27-G8", "P30-G4", "P31-G5", "P32-G6",
     "P33-G7", "P53-G8", "P55-G1", "P58-G5", "P59-G8", "P60-G6", "P61-G4", "P62-G3", "P63-G5", 
     "P65-G1", "P66-G5", "P67-G3", "P69-G8", "P72-G1", "P73-G3", "P74-G1", "P75-G1"]  # list of str "Pxx-Gy", e.g. ["P02-G2"]

TASKS: list[str]                    = ["oddball","UT"]  # ["oddball"], ["UT"] or ["oddball", "UT"]
RUNS: list[int]                     = [1,2]  # [1], [2] or [1, 2]

inputs = [
    (t[0].split("-") + list(t[1:]))
    for t in product(PARTICIPANTS_WITH_GROUPS, TASKS, RUNS)
]
print(inputs)

[['P05', 'G2', 'oddball', 1], ['P05', 'G2', 'oddball', 2], ['P05', 'G2', 'UT', 1], ['P05', 'G2', 'UT', 2], ['P07', 'G5', 'oddball', 1], ['P07', 'G5', 'oddball', 2], ['P07', 'G5', 'UT', 1], ['P07', 'G5', 'UT', 2], ['P08', 'G6', 'oddball', 1], ['P08', 'G6', 'oddball', 2], ['P08', 'G6', 'UT', 1], ['P08', 'G6', 'UT', 2], ['P10', 'G8', 'oddball', 1], ['P10', 'G8', 'oddball', 2], ['P10', 'G8', 'UT', 1], ['P10', 'G8', 'UT', 2], ['P11', 'G1', 'oddball', 1], ['P11', 'G1', 'oddball', 2], ['P11', 'G1', 'UT', 1], ['P11', 'G1', 'UT', 2], ['P12', 'G2', 'oddball', 1], ['P12', 'G2', 'oddball', 2], ['P12', 'G2', 'UT', 1], ['P12', 'G2', 'UT', 2], ['P13', 'G3', 'oddball', 1], ['P13', 'G3', 'oddball', 2], ['P13', 'G3', 'UT', 1], ['P13', 'G3', 'UT', 2], ['P14', 'G3', 'oddball', 1], ['P14', 'G3', 'oddball', 2], ['P14', 'G3', 'UT', 1], ['P14', 'G3', 'UT', 2], ['P15', 'G4', 'oddball', 1], ['P15', 'G4', 'oddball', 2], ['P15', 'G4', 'UT', 1], ['P15', 'G4', 'UT', 2], ['P16', 'G4', 'oddball', 1], ['P16', 'G4', 'o

The variable `inputs` contains is a list of list. Each sublist defines one file by its participant, group, task and run attribute. Each sublist is one set of input variable for `apply_ica` which will be picked up by a worker (process) and executed. For each execution, the created derivatives are:
- Preprocessed raw recording (`_step10_preprocessed_raw.fif`)

This step ends with a visual inspection of the raw recording during which `BAD` segments annotations should be edited based on what was well corrected and what was not.

In [ ]:
%%time
current_time = time.strftime("%H:%M:%S")
print("Start time", current_time)

assert len(inputs) != 0  # sanity-check
# each job will need 1 threads and mp.cpu_count() usually returns
# the number of threads (usually 2 per core), thus let's spawn 2
# process per core maximum.
n_jobs = min(len(inputs), (mp.cpu_count() - 2))
parallel(apply_ica_interpolate, n_jobs, inputs)

Start time 16:56:28


 39%|███████████████████████████████▍                                                 | 59/152 [09:35<12:27,  8.03s/it]